In [ ]:
!cp -r ../input/recsysgit/RecSys_Course_AT_PoliMi-master/* ./

In [ ]:
%config Completer.use_jedi = False
%load_ext Cython
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

In [ ]:
!python run_compile_all_cython.py

In [ ]:
#!zip -r -j /kaggle/working/SLIMElasticNetRecommender.zip /kaggle/input/slim-ema/W_sparse.npz

In [ ]:
URM_all_dataframe = pd.read_csv('../input/urmema/URM_EMA.csv')
URM_all_dataframe

In [ ]:
URM_all = sps.coo_matrix((URM_all_dataframe["Data"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)
URM_all

In [ ]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

# split data into train and validation data 80/20
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout

#create an evaluator object to evaluate validation set
#we will use it for hyperparameter tuning
evaluator_valid = EvaluatorHoldout(URM_valid, cutoff_list=[10])

In [ ]:
import numpy as np
import scipy.sparse as sps
from Recommenders.Recommender_utils import check_matrix
from sklearn.linear_model import ElasticNet
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit
import time, sys
from tqdm import tqdm
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# os.environ["PYTHONWARNINGS"] = ('ignore::exceptions.ConvergenceWarning:sklearn.linear_model')
# os.environ["PYTHONWARNINGS"] = ('ignore:Objective did not converge:ConvergenceWarning:')

class SLIMElasticNetRecommender(BaseItemSimilarityMatrixRecommender):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available
    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.
        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train, verbose = True):
        super(SLIMElasticNetRecommender, self).__init__(URM_train, verbose = verbose)

    @ignore_warnings(category=ConvergenceWarning)
    def fit(self, l1_ratio=0.1, alpha = 1.0, positive_only=True, topK = 100,**earlystopping_kwargs):

        assert l1_ratio>= 0 and l1_ratio<=1, "{}: l1_ratio must be between 0 and 1, provided value was {}".format(self.RECOMMENDER_NAME, l1_ratio)

        self.l1_ratio = l1_ratio
        self.positive_only = positive_only
        self.topK = topK


        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=alpha,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)

        URM_train = check_matrix(self.URM_train, 'csc', dtype=np.float32)

        n_items = URM_train.shape[1]

        # Use array as it reduces memory requirements compared to lists
        dataBlock = 10000000

        rows = np.zeros(dataBlock, dtype=np.int32)
        cols = np.zeros(dataBlock, dtype=np.int32)
        values = np.zeros(dataBlock, dtype=np.float32)

        numCells = 0

        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0

            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values

            # Select topK values
            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index

            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data

            local_topK = min(len(nonzero_model_coef_value)-1, self.topK)

            relevant_items_partition = (-nonzero_model_coef_value).argpartition(local_topK)[0:local_topK]
            relevant_items_partition_sorting = np.argsort(-nonzero_model_coef_value[relevant_items_partition])
            ranking = relevant_items_partition[relevant_items_partition_sorting]

            for index in range(len(ranking)):

                if numCells == len(rows):
                    rows = np.concatenate((rows, np.zeros(dataBlock, dtype=np.int32)))
                    cols = np.concatenate((cols, np.zeros(dataBlock, dtype=np.int32)))
                    values = np.concatenate((values, np.zeros(dataBlock, dtype=np.float32)))


                rows[numCells] = nonzero_model_coef_index[ranking[index]]
                cols[numCells] = currentItem
                values[numCells] = nonzero_model_coef_value[ranking[index]]

                numCells += 1

            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup

            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Items per second: {:.2f}".format(
                    currentItem+1,
                    100.0* float(currentItem+1)/n_items,
                    new_time_value,
                    new_time_unit,
                    float(currentItem)/elapsed_time))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        # generate the sparse weight matrix
        self.W_sparse = sps.csr_matrix((values[:numCells], (rows[:numCells], cols[:numCells])),
                                       shape=(n_items, n_items), dtype=np.float32)

In [ ]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [ ]:
recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_train)
recommender_SLIMElasticNet.fit(epochs = 500, l1_ratio = 0.001, alpha = 0.01, 
               positive_only = True, topK = 384)

In [ ]:
RP3beta_all = RP3betaRecommender(URM_train)
RP3beta_all.fit(alpha = 1.0, beta = 0.3553908871486053, topK = 74, implicit = True)

In [ ]:
print("MAP of the starting models")

result_df, _ = evaluator_valid.evaluateRecommender(recommender_SLIMElasticNet)
print("SLIM - MAP: {}".format(result_df.loc[10]["MAP"]))

result_df, _ = evaluator_valid.evaluateRecommender(RP3beta_all)
print("RP3beta - MAP: {}".format(result_df.loc[10]["MAP"]))

In [ ]:
alpha = 0.45
new_similarity = (1 - alpha) * recommender_SLIMElasticNet.W_sparse + alpha * RP3beta_all.W_sparse

Merged_recommender = ItemKNNCustomSimilarityRecommender(URM_train)
Merged_recommender.fit(new_similarity)

**LINEAR COMBINATION BETWEEN P3ALPHA AND MERGED RECOMMENDER (RP3BETA+SIM-EN)**

In [ ]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
        
    def fit(self, norm, alpha = 0.5):

        self.alpha = alpha
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (1-self.alpha)

        return item_weights

**TUNING OF AALPHA AND NORM, SKIP WHEN GENERATING RECOMMENDATIONS**

In [ ]:
EASE_R = EASE_R_Recommender(URM_train)
#%%
EASE_R.fit(topK = 416, l2_norm = 115.67139771839786, normalize_matrix = False)

In [ ]:
print("MAP of the starting models")

result_df, _ = evaluator_valid.evaluateRecommender(Merged_recommender)
print("SLIM MERGED - MAP: {}".format(result_df.loc[10]["MAP"]))

result_df, _ = evaluator_valid.evaluateRecommender(EASE_R)
print("EASE_R - MAP: {}".format(result_df.loc[10]["MAP"]))

In [ ]:
"""
recommender_object = DifferentLossScoresHybridRecommender(URM_train, Merged_recommender, EASE_R)

best_model = {
    "MAP" : 0,
    "alpha" : 0,
    "norm" : 0
}

for norm in [1, 2]:
    for alpha in np.arange(0,1,0.1):
        print("----")
        recommender_object.fit(norm, alpha)

        result_df, _ = evaluator_valid.evaluateRecommender(recommender_object)
        print("Norm: {}, Alpha: {}, Result: {}".format(norm, alpha, result_df.loc[10]["MAP"]))

        if result_df.loc[10]["MAP"] > best_model["MAP"]:
            best_model["MAP"] = result_df.loc[10]["MAP"]
            best_model["alpha"] = alpha
            best_model["norm"] = norm
print("----")
print("Best model has MAP: {} with alpha: {}, norm: {}".format(best_model["MAP"], best_model["alpha"], best_model["norm"]))

"""

In [ ]:
recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_all)
recommender_SLIMElasticNet.fit(epochs = 500, l1_ratio = 0.001, alpha = 0.01, 
               positive_only = True, topK = 384)

In [ ]:
RP3beta_all = RP3betaRecommender(URM_all)
RP3beta_all.fit(alpha = 1.0, beta = 0.3553908871486053, topK = 74, implicit = True)

In [ ]:
alpha = 0.45
new_similarity = (1 - alpha) * recommender_SLIMElasticNet.W_sparse + alpha * RP3beta_all.W_sparse

Merged_recommender = ItemKNNCustomSimilarityRecommender(URM_all)
Merged_recommender.fit(new_similarity)

**FITTING EASE_R AND HYBRID RECOMMENDERS ON URM_ALL**

In [ ]:
EASE_R = EASE_R_Recommender(URM_all)
#%%
EASE_R.fit(topK = 416, l2_norm = 115.67139771839786, normalize_matrix = False)

In [ ]:
recommender = DifferentLossScoresHybridRecommender(URM_all, Merged_recommender, EASE_R)
recommender.fit(1, 0.6)

In [ ]:
test_users = pd.read_csv('/kaggle/input/recsyschallenge/data_target_users_test.csv')
test_users

In [ ]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user,cutoff = 10))
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('submission.csv', index=False)